<img src='https://albumentations.ai/docs/images/logo.png' width='160'>


<h1><center>[Tutorial] Albumentations with masks</center><h1>
    
# <a id='1'>Let's start augmentation with masks🔥 </a>

- Version `13` : Add `CoarseDropout with Masks` and more examples
- Version `8` : Add basic guideline and simple examples

## If this kernel is useful, <font color='orange'>please upvote</font>!

# What is Albumentations

`Albumentations` is a Python library for fast and flexible image augmentations. 

Albumentations efficiently implements a rich variety of image transform operations that are optimized for performance, and does so while providing a concise, yet powerful image augmentation interface for different computer vision tasks, including object classification, segmentation, and detection.

<img src='https://camo.githubusercontent.com/3bb6e4bb500d96ad7bb4e4047af22a63ddf3242a894adf55ebffd3e184e4d113/68747470733a2f2f686162726173746f726167652e6f72672f776562742f62642f6e652f72762f62646e6572763563746b75646d73617a6e687734637273646669772e6a706567' width='640'>


## New feature - releases 0.5.1 

`A.ToTensorV2` now supports an additional argument transpose_mask (False by default).

If the argument is set to True and an input mask has 3 dimensions, A.ToTensorV2 will transpose dimensions of a mask tensor in addition to transposing dimensions of an image tensor.



# Installation

In [ ]:
!pip install -U git+https://github.com/albumentations-team/albumentations > /dev/null 2>&1

In [ ]:
import os
import cv2
import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

# Dataset

https://www.kaggle.com/orkatz2/hubmap-res34unet-baseline-train

In [ ]:
images_path = '../input/hubmap-256x256/train'
masks_path = '../input/hubmap-256x256/masks'

In [ ]:
class HuBMAPDataset(Dataset):
    def __init__(self, ids, transforms=None, preprocessing=None):
        self.ids = ids
        self.transforms = transforms
        self.preprocessing = preprocessing
    def __getitem__(self, idx):
        name = self.ids[idx]
        img = cv2.imread(f"{images_path}/{name}")
        mask = cv2.imread(f"{masks_path}/{name}")[:,:,0:1]
        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        '''
        # Now, we are use new feature in albumentations
        if self.preprocessing:
            preprocessed = self.preprocessing(image=img, mask=mask)
            img = preprocessed['image']
            mask = preprocessed['mask']
        '''
        return img, mask

    def __len__(self):
        return len(self.ids)

# Albumentations with masks

Use `transpose_mask`=`True`.

In [ ]:
def get_train_augmentation(size=1024):
    return A.Compose([
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.ShiftScaleRotate(),
        A.CoarseDropout(max_holes=8, max_height=20, max_width=20, mask_fill_value=0, always_apply=True), # For visualization, set always_apply=True.
        A.Resize(size,size, always_apply=True),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),
        ## Check transpose_mask=True
        ToTensorV2(transpose_mask=True)
    ])

def get_valid_augmentation(size=1024):
    return A.Compose([
        A.Resize(size,size, always_apply=True),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),
        ## Check transpose_mask=True
        ToTensorV2(transpose_mask=True)
    ])

In [ ]:
data = os.listdir(images_path)
train_lsit = list(set([row.split("_")[0] for row in data]))
train_idx = [row for row in data if row.split("_")[0] in train_lsit[:-2]]
valid_idx = [row for row in data if row.split("_")[0] not in train_lsit[:-2]]
len(train_idx),len(valid_idx)

In [ ]:
train_aug = get_train_augmentation(size=1024)
train_dataset = HuBMAPDataset(
    ids=train_idx,
    transforms=train_aug)

# Applying augmentation with mask

## First
Focus on that image and compare second one.

We can see also `CoarseDropout` results between image and mask.

In [ ]:
# I pick train_dataset[11] for mask
img, mask = train_dataset[11]
img = img.permute(1,2,0).detach().numpy()
mask = mask.permute(1,2,0).detach().numpy()

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(img)
ax[1].imshow(mask[:,:,0])

## Second

In [ ]:
# I pick train_dataset[11] for mask
img, mask = train_dataset[11]
img = img.permute(1,2,0).detach().numpy()
mask = mask.permute(1,2,0).detach().numpy()

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(img)
ax[1].imshow(mask[:,:,0])

Different augmentations **are applied to** `image` and `mask`!

# Multiple View

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(
    train_dataset, 
    batch_size=16,
    collate_fn=collate_fn
)

In [ ]:
n_rows=4
n_cols=4

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)

images, masks = next(iter(train_loader))

# plot some augmentations!
fig, ax = plt.subplots(figsize=(20, 20),  nrows=n_rows, ncols=n_cols)
for i in range (n_rows*n_cols):     
    image = images[i].permute(1,2,0).detach().numpy()
    mask = masks[i].permute(1,2,0).detach().numpy()

    ax[i // n_rows][i % n_cols].imshow(image)
    ax[i // n_rows][i % n_cols].imshow(mask[:,:,0], cmap="hot", alpha=0.7)

# Let's start more Albumentation with masks

https://www.kaggle.com/iafoss/hubmap-pytorch-fast-ai-starter

## Base Image and mask

I used `train_dataset[11]` for visualization.

In [ ]:
dataset_index = 11

In [ ]:
valid_aug = get_valid_augmentation(size=1024)
train_dataset = HuBMAPDataset(
    ids=train_idx,
    transforms=valid_aug)

img, mask = train_dataset[dataset_index]

img = img.permute(1,2,0).detach().numpy()
mask = mask.permute(1,2,0).detach().numpy()

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(img)
ax[1].imshow(mask[:,:,0])

## HorizontalFlip

In [ ]:
transformed_mask = A.HorizontalFlip(always_apply=True)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## VerticalFlip

In [ ]:
transformed_mask = A.VerticalFlip(always_apply=True)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## RandomRotate90

In [ ]:
transformed_mask = A.RandomRotate90(always_apply=True)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## ShiftScaleRotate

In [ ]:
transformed_mask = A.ShiftScaleRotate(shift_limit=0.25, 
                                      scale_limit=0.25, 
                                      rotate_limit=15, 
                                      border_mode=cv2.BORDER_REFLECT,
                                      always_apply=True)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## OpticalDistortion

- Parameters such as `distort_limit`, `shift_limit` are exaggerated for visualization.
- Be careful of use.

In [ ]:
transformed_mask = A.OpticalDistortion(always_apply=True, 
                                       distort_limit=0.85, 
                                       shift_limit=0.85, 
                                       mask_value=0)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## GridDistortion

- Parameters such as `distort_limit` are exaggerated for visualization.
- Be careful of use.

In [ ]:
transformed_mask = A.GridDistortion(always_apply=True, 
                                    distort_limit=0.85, 
                                    mask_value=0)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## ElasticTransform

In [ ]:
transformed_mask = A.ElasticTransform(alpha=120, 
                                      sigma=120 * 0.05, 
                                      alpha_affine=120 * 0.03, 
                                      always_apply=True,
                                      mask_value=0)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## CoarseDropout

In [ ]:
transformed_mask = A.CoarseDropout(max_holes=8, 
                                   max_height=50, 
                                   max_width=50, 
                                   mask_fill_value=0, 
                                   always_apply=True)(image=mask)

fig, ax = plt.subplots(figsize=(16, 8),  nrows=1, ncols=2)
ax[0].imshow(mask[:,:,0])
ax[1].imshow(transformed_mask['image'][:,:,0])

## If this kernel is useful, <font color='orange'>please upvote</font>!
- See you next time!